In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.regularizers import l2
from keras.optimizers import SGD ,Adagrad
from scipy.io import loadmat, savemat
from keras.models import model_from_json
import theano.tensor as T
import theano
import csv
import configparser
import collections
import time
import csv
import os
from os import listdir
import skimage.transform
from skimage import color
from os.path import isfile, join
import numpy as np
import numpy
from datetime import datetime
from scipy.spatial.distance import cdist,pdist,squareform
import theano.sandbox
import shutil

In [23]:
seed = 7
numpy.random.seed(seed)


def load_model(json_path):  # Function to load the model
    model = model_from_json(open(json_path).read())
    return model

def load_weights(model, weight_path):  # Function to load the model weights
    dict2 = loadmat(weight_path)
    dict = conv_dict(dict2)
    i = 0
    for layer in model.layers:
        weights = dict[str(i)]
        layer.set_weights(weights)
        i += 1
    return model

def conv_dict(dict2):
    i = 0
    dict = {}
    for i in range(len(dict2)):
        if str(i) in dict2:
            if dict2[str(i)].shape == (0, 0):
                dict[str(i)] = dict2[str(i)]
            else:
                weights = dict2[str(i)][0]
                weights2 = []
                for weight in weights:
                    if weight.shape in [(1, x) for x in range(0, 5000)]:
                        weights2.append(weight[0])
                    else:
                        weights2.append(weight)
                dict[str(i)] = weights2
    return dict

In [24]:
# Load Video

def load_dataset_One_Video_Features(Test_Video_Path):

    VideoPath =Test_Video_Path
    f = open(VideoPath, "r")
    words = f.read().split()
    num_feat = int(len(words) / 4096)
    # Number of features per video to be loaded. In our case num_feat=32, as we divide the video into 32 segments. Note that
    # we have already computed C3D features for the whole video and divided the video features into 32 segments.
    print(num_feat)
    count = -1;
    VideoFeatues = []
    for feat in range(0, num_feat):
        feat_row1 = np.float32(words[feat * 4096:feat * 4096 + 4096])
        count = count + 1
        if count == 0:
            VideoFeatues = feat_row1
        if count > 0:
            VideoFeatues = np.vstack((VideoFeatues, feat_row1))
    AllFeatures = VideoFeatues

    return  AllFeatures


In [25]:
print("Starting testing...")


AllTest_Video_Path = './Test2'
# AllTest_Video_Path contains C3D features (txt file)  of each video. Each file contains 32 features, each of 4096 dimensions.
Results_Path = './Eval_Res/'
# Results_Path is the folder where you can save your results
Model_dir='./'
# Model_dir is the folder where we have placed our trained weights
weights_path = Model_dir + 'weights_L1L2.mat'
# weights_path is Trained model weights

model_path = Model_dir + 'model.json'

if not os.path.exists(Results_Path):
       os.makedirs(Results_Path)

All_Test_files= [i for i in listdir(AllTest_Video_Path) if i[0] != '.']
All_Test_files.sort()

Starting testing...


In [26]:
All_Test_files

['finput.txt']

In [27]:
model=load_model(model_path)
load_weights(model, weights_path)
nVideos=len(All_Test_files)
time_before = datetime.now()
result = []
for iv in range(nVideos):

    Test_Video_Path = os.path.join(AllTest_Video_Path, All_Test_files[iv])
    inputs=load_dataset_One_Video_Features(Test_Video_Path) # 32 segments features for one testing video
    predictions = model.predict_on_batch(inputs)   # Get anomaly prediction for each of 32 video segments.
    aa=All_Test_files[iv]
    aa=aa[0:-4]
    A_predictions_path = Results_Path + aa + '.mat'  # Save array of 1*32, containing anomaly score for each segment. Please see Evaluate Anomaly Detector to compute  ROC.
    print ("Total Time took: " + str(datetime.now() - time_before))
    result.append(predictions)

32
Total Time took: 0:00:00.139573


In [19]:
import pandas as pd

In [20]:
data = pd.read_csv('1Explosion008_x264_000866.csv')

In [28]:
np.array(result)

array([[[6.0897606e-04],
        [1.3030178e-03],
        [7.8127280e-02],
        [9.9751425e-01],
        [9.4574887e-01],
        [4.0951747e-01],
        [2.8301280e-02],
        [1.1295226e-03],
        [2.2240597e-04],
        [1.7030432e-05],
        [1.5427113e-06],
        [2.1062846e-07],
        [9.7279120e-08],
        [4.7137394e-08],
        [1.5695929e-07],
        [4.7346130e-06],
        [6.4792024e-05],
        [1.9794323e-04],
        [1.1276546e-03],
        [3.8371286e-03],
        [1.6858036e-02],
        [7.7868536e-02],
        [6.7855924e-02],
        [1.0964286e-01],
        [1.6335015e-01],
        [2.2282764e-01],
        [1.5309086e-01],
        [2.2730345e-01],
        [4.7287855e-01],
        [5.8449858e-01],
        [7.4868846e-01],
        [7.8337359e-01]]], dtype=float32)